---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [37]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [38]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [ ]:
'''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''

In [3]:
def get_list_of_university_towns():
    state= None
    States=[]
    county=None
    with open('university_towns.txt') as file:
        for line in file:
            xline=line[:-1]
            if xline[-6:]=='[edit]':
                state=xline[:-6]
                continue
            if '(' in xline:
                county=xline[:xline.index('(')-1]
                States.append([state,county])
            else:
                States.append([state,xline.strip()])
    df=pd.DataFrame(States,columns=['State','RegionName'])
          
            
            
   
    return df
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [26]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df1=pd.read_excel('gdplev.xls',skiprows=219)
    df1=df1[['1999q4', 9926.1]]
    df1.columns=['Quarter','gdp']
  #  df1.set_index('Quarter',inplace=True)
    for i in range(2,len(df1)):
        if (df1.iloc[i-2][1]>df1.iloc[i-1][1] and df1.iloc[i-1][1]>df1.iloc[i][1]):
            return df1.iloc[i-2]['Quarter']
            
    
    
    
get_recession_start()

'2008q3'

In [27]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    df1=pd.read_excel('gdplev.xls',skiprows=219)
    df1=df1[['1999q4', 9926.1]]
    df1.columns=['Quarter','gdp']
    for i in range(2,len(df1)-2):
        if (df1.iloc[i-2][1]>df1.iloc[i-1][1] and df1.iloc[i-1][1]>df1.iloc[i][1])and (df1.iloc[i+2][1]>df1.iloc[i+1][1] and df1.iloc[i+1][1]>df1.iloc[i][1]):
            return df1.iloc[i+2]['Quarter']
    
       
get_recession_end()   

'2009q4'

In [39]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df1=pd.read_excel('gdplev.xls',skiprows=219)
    df1=df1[['1999q4', 9926.1]]
    df1.columns=['Quarter','gdp']
    df1.set_index('Quarter',inplace=True)
    start=get_recession_start()
    end=get_recession_end()
    df1=df1.loc[start:end]
    
    return df1.idxmin()[0]
get_recession_bottom()

'2009q2'

In [ ]:
'''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. 
    
    
   
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''

In [22]:
def convert_housing_data_to_quarters():
    df2=pd.read_csv('City_Zhvi_AllHomes.csv')
    df2.drop(['Metro','CountyName','RegionID','SizeRank'],axis=1,inplace=1)
    df2['State'] = df2['State'].map(states)
    #df2.set_index(['State','RegionName'],inplace=True)
    #df2.drop(list(df2.columns)[0:45],inplace=1,axis=1)
    dfx=pd.DataFrame(df2[['State','RegionName']])
    for year in range(2000,2016):
        dfx[str(year)+'q1']=df2[[str(year)+'-01',str(year)+'-02',str(year)+'-03']].mean(axis=1)
        dfx[str(year)+'q2']=df2[[str(year)+'-04',str(year)+'-05',str(year)+'-06']].mean(axis=1)
        dfx[str(year)+'q3']=df2[[str(year)+'-07',str(year)+'-08',str(year)+'-09']].mean(axis=1)
        dfx[str(year)+'q4']=df2[[str(year)+'-10',str(year)+'-11',str(year)+'-12']].mean(axis=1)
    year=2016    
    dfx[str(year)+'q1']=df2[[str(year)+'-01',str(year)+'-02',str(year)+'-03']].mean(axis=1)
    dfx[str(year)+'q2']=df2[[str(year)+'-04',str(year)+'-05',str(year)+'-06']].mean(axis=1)
    dfx[str(year)+'q3']=df2[[str(year)+'-07',str(year)+'-08']].mean(axis=1)
    dfx.set_index(['State','RegionName'],inplace=True)
    

    
    return dfx
convert_housing_data_to_quarters()

(10730, 67)

In [ ]:
'''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''

In [ ]:
def run_ttest():
    df=convert_housing_data_to_quarters().copy()
    start=get_recession_start()
    end=get_recession_end()
    bottom=get_recession_bottom()
    df=df.loc[:,start:end]
    df.reset_index(inplace=True)
    #def price_ratio(row):
    #    return (row['2008q3'] - row['2009q2'])/row['2008q3']
    
    #df['up&down'] = df.apply(price_ratio,axis=1)
    df['updown']=(df['2008q3'] - df['2009q2'])/df['2008q3']
    uni = get_list_of_university_towns()['RegionName']
    uni = set(uni)

    def is_uni_town(row):
       
        if row['RegionName'] in uni:
            return 1
        else:
            return 0
    df['isuni']=df.apply(is_uni_town,axis=1)  
    notu = df[df['isuni']==0].loc[:,'updown'].dropna()
    isu = df[df['isuni']==1].loc[:,'updown'].dropna()
    z=None
    if isu.mean()<notu.mean():
        z='university town'
    else:
        z='non-university town'
    p=list(ttest_ind(notu,isu))[1]
    
    
    return (True,p,z)
run_ttest()